In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import numpy as np
import skimage as sk
import skimage.io as io
import matplotlib
%matplotlib inline
from skimage.util import img_as_ubyte
import utils, morph

from main import *

In [62]:
from scipy.spatial import Delaunay
from scipy.interpolate import interp2d

In [63]:
from pathlib import Path
data = Path("input")
data.mkdir(parents=True, exist_ok=True)

In [64]:
# matplotlib.use("TkAgg")
# matplotlib.use("Tk")
# matplotlib.interactive(True)

In [65]:
me = utils.read_img("src_images/me.jpg")
me.shape
# io.imshow(me)

(2640, 1980, 3)

In [66]:
iu = utils.read_img("src_images/iu_4.jpg")
iu.shape

(1071, 900, 3)

In [78]:
%matplotlib tk
#me_aligned = utils.align_img("input/me.jpg")
iu_aligned = utils.align_img("input/iu.jpg")
# io.imsave("input/me.jpg", img_as_ubyte(me_aligned), format="jpg",)
# io.imsave("input/iu.jpg", img_as_ubyte(iu_aligned), format="jpg",)

range as ubyte: (0, 255)
range as float: (0.0, 1.0)
max 1.0
0 0.0 1.0
1 0.0001344495855001728 1.0


In [ ]:
%matplotlib inline
io.imshow(me_aligned)